In [ ]:
class TaggedDocumentIterator(object):

  def __init__(self, doc_list, labels_list):
      self.labels_list = labels_list
      self.doc_list = doc_list

  def __iter__(self):
      for idx, doc in enumerate(self.doc_list):
        for idy in enumerate(doc):
          yield TaggedDocument(words=idy[1].split(), tags=[str(self.labels_list[idx]) + '.' + str(doc.index(idy[1]))])


In [ ]:
docLabels = list(range(len(paper_df)))
doc_list = list(paper_df)
tag_doc_iter = TaggedDocumentIterator(doc_list, docLabels)

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Doc2Vec model training
def train_doc2vec(papers, tagged_data):
    #tagged_data = [TaggedDocument(words=word_tokenize(paper.lower()), tags=[str(i)]) for i, paper in enumerate(papers)]

    model = Doc2Vec(vector_size=300, window=8, min_count=1, workers=8, epochs=100, alpha=0.025, min_alpha=0.025)
    #vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8
    #vector_size=100, window=4, min_count=1, workers=8, epochs=100

    model.build_vocab(tagged_data)

    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

    return model, tagged_data

In [ ]:
def modst_similar(model, new_abstract):
  # vector extraction about new abstract
  new_vector = model.infer_vector(word_tokenize(new_abstract.lower()))

  # 각 논문과의 유사도 계산
  similarities = model.dv.cosine_similarities(new_vector, model.wv.vectors)

  # 유사도에 따라 정렬
  sorted_similarities = sorted(enumerate(similarities), key=lambda x: x[1], reverse=True)

  # 가장 유사한 논문 출력
  top_similar_paper_idx, top_similarity = sorted_similarities[0]
  top_similar_paper = paper_df.iloc[top_similar_paper_idx]
  print(f"Most similar paper:")
  print(f"Title: {top_similar_paper['Title']}")
  print(f"Abstract: {top_similar_paper['Abstract']}")



---



In [ ]:
model, tagged_data = train_doc2vec(paper_df, tag_doc_iter)

In [ ]:
# new abstract

new_abstract = 'Alzheimer’s disease (AD) is the most common late‐onset neurodegenerative disorder. \
Identifying individuals at increased risk of developing AD is important for early intervention. \
Using data from the Alzheimer Disease Genetics Consortium, we constructed polygenic risk scores (PRSs) for AD and age‐ at‐onset (AAO) of AD for the UK Biobank participants. \
We then built machine learning (ML) models for predicting development of AD, and explored feature importance among PRSs, conventional risk factors, \
and ICD‐10 codes from electronic health records, a total of > 11,000 features using the UK Biobank dataset. \
We used eXtreme Gradient Boosting (XGBoost) and SHapley Additive exPlanations (SHAP), which provided superior ML performance as well as aided ML model explanation. \
For participants age 40 and older, the area under the curve for AD was 0.88. For subjects of age 65 and older (late‐onset AD), PRSs were the most important predictors. \
This is the first observation that PRSs constructed from the AD risk and AAO play more important roles than age in predicting AD. \
The ML model also identified important predictors from EHR, including urinary tract infection, syncope and collapse, chest pain, disorientation and hypercholesterolemia, for developing AD. \
Our ML model improved the accuracy of AD risk prediction by efficiently exploring numerous predictors and identified novel feature patterns.'

"""
new_abstract = "NFTrig is a web-based application created for use as an educational tool to teach trigonometry and block chain technology. \
Creation of the application includes front and back end development as well as integration with other outside sources including MetaMask and OpenSea. \
The primary development languages include HTML, CSS (Bootstrap 5), and JavaScript as well as Solidity for smart contract creation. \
The application itself is hosted on Moralis utilizing their Web3 API. \
This technical report describes how the application was created, what the application requires, and smart contract design with security considerations in mind. \
The NFTrig application has underwent significant testing and validation prior to and after deployment. \
Future suggestions and recommendations for further development, maintenance, and use in other fields for education are also described."
"""

In [ ]:
most_paper = modst_similar(model, new_abstract)

In [ ]:
def similarity_paper_extraction(new_abstract, model):
  # vector extraction about new abstract
  new_vector = model.infer_vector(word_tokenize(new_abstract.lower()))

  # 각 논문과의 유사도 계산
  similarities = model.dv.cosine_similarities(new_vector, model.wv.vectors)

  # 유사도에 따라 정렬
  sorted_similarities = sorted(enumerate(similarities), key=lambda x: x[1], reverse=True)

  similar_papers = []
  for idx, similarity in sorted_similarities:
      top_similar_paper = paper_df.iloc[idx]
      similar_papers.append(top_similar_paper)
      #print(f"Similarity: {similarity:.4f}")
      #print(f"Title: {top_similar_paper['Title']}")
      #print(f"Abstract: {top_similar_paper['Abstract']}")
      #print()

  similar_paper_df = pd.DataFrame(similar_papers)
  #similar_papers_df.to_csv('similar_papers.csv', index=False)
  similar_paper_df

  return sorted_similarities, similar_paper_df

In [ ]:
sorted_similarities, similar_paper_df = similarity_paper_extraction(new_abstract, model)

In [ ]:
similar_paper_df

In [ ]:
sorted_similarities

In [1]:
# test 2023-11-23